<a href="https://colab.research.google.com/github/aquapathos/ColabHowto/blob/master/%E5%A4%96%E9%83%A8%E3%81%8B%E3%82%89_Colab_%E3%83%9E%E3%82%B7%E3%83%B3%E5%86%85%E3%81%AE%E3%82%B5%E3%83%BC%E3%83%93%E3%82%B9%E3%81%AB%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorboard

Tensorboard は、

In [4]:
# ngrokのインストール
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

# tensorboard のインストール
!pip install tensorboard==1.14.0 > /dev/null
!pip install tb-nightly > /dev/null

--2020-11-28 03:18:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.1.26.21, 52.22.13.178, 3.226.231.47, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.1.26.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  56.6MB/s    in 0.2s    

2020-11-28 03:18:04 (56.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# ランタイムを再起動してから、下に進む

In [5]:
import os
# TensorBoardのlogフォルダ
LOG_DIR = './log'
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

# TensorBoardをバックグラウンドで実行
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

# ngrokバックグラウンドプロセスを起動
# TensorBoardのポート6006を開放します。
get_ipython().system_raw('./ngrok http 6006 &')

In [6]:
# TensorBoard表示のURLを取得
# localhost:4040でNgrokのステータスをJsonで取得してpublicUrlをPrintしている
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://c74265fcae8e.ngrok.io


# プログラムで 上の　url にアクセスするための方法
上のプログラムはマジックコマンドで実行しているので、url　は表示されているが、 python　のプログラムで使えない。

## 方法１
url をテキストファイルとして書き出し、後でそのファイルを読み出す。

In [7]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json;url = json.load(sys.stdin)['tunnels'][0]['public_url'];print(url);f = open('url.txt', 'w');f.write(url);f.close()"

https://c74265fcae8e.ngrok.io


これで、url のみが書かれた、'url.txt'　というファイルができるので、後でこのファイルから読み出せばいい。


In [10]:
# get ngrok url
f = open("url.txt", "r")
url = f.read()
f.close()
print("url={}".format(url))

url=https://c74265fcae8e.ngrok.io


## 方法２

subprosess.run で、外部コマンドを実行したときの stdout を取り出して、そこから直接 url を取得する

In [8]:
import subprocess
result = subprocess.run(["curl -s http://localhost:4040/api/tunnels | python3 -c \"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])\""], encoding='utf-8',shell=True, stdout=subprocess.PIPE)
url = result.stdout[:-1]
print("url={}".format(url))

url=http://c74265fcae8e.ngrok.io
